In [138]:
import torch
import pandas
import matplotlib.pyplot as plt


In [139]:
import random

Exericise 1: Building a trigram lm

Q : Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model? 

In [140]:
file=open('/content/drive/MyDrive/building_makemore/names.txt')

In [141]:
words=file.read().split('\n')

In [142]:
len(words)

32033

In a trigram lm, we will need 2 characters in the predictor used to predict the next character unlike a bigram in which we needed only one previous character

.em
emm
mma
ma.

In [143]:
for word in words[:1]:
  chars=['.']+list(word)+['.']

  for f,s,t in zip(chars,chars[1:],chars[2:]):
    print(f,s,t)

. e m
e m m
m m a
m a .


word "emma" specifically would contribute 4 training samples.

Building the training set using the entire corpus and segregating the the first two characters into xs and the 3rd character into ys

In [144]:
# First convert characters into integers by building a stoi and itos dictionary

# xs=[]
# ys=[]

# for word in words:
#   chars=['.']+list(word)+['.']

#   for f,s,t in zip(chars,chars[1:],chars[2:]):
#     xs.append()
#     ys.append()



Ok, i realised that the predictor array i will have will contain characters. First we need a way to change characters into integers


In [145]:
# Building the dictionary- just a sample on how to proceed

for i,c in enumerate(sorted(list(set(''.join(words[:2]))))):
  print(i,c)

0 a
1 e
2 i
3 l
4 m
5 o
6 v


In [146]:
stoi={c:i+1 for i,c in enumerate(sorted(list(set(''.join(words)))))}
stoi['.']=0
itos={i:c for c,i in stoi.items()}


In [147]:
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [148]:
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

Lets build it using manual method first i.e using counts matrix

In [149]:
# This will store the counts of each trigram which occurs in our dataset
N=torch.zeros((27,27,27),dtype=torch.int32)

for word in words:
  chars=['.']+list(word)+['.']

  for f,s,t in zip(chars,chars[1:],chars[2:]):
    ch1=stoi[f]
    ch2=stoi[s]
    ch3=stoi[t]
    N[ch1,ch2,ch3]+=1
    

******

Rough work- checks

In [150]:
# This basically gives the counts of trigrams which start with '.' and 'a' with 3rd character chosen from the 27 characters
N[0,1]

tensor([  0, 207, 190,  31, 366,  55,  21,  17,  91, 154,  27,  75, 632, 384,
        623,  10,  17,   9, 482, 194,  72, 152, 243,   6,  27, 173, 152],
       dtype=torch.int32)

In [151]:
# This basically sums all counts and will be used in normalising the values of N[0,1] to give us probabilities
N[0,1].sum()

tensor(4410)

******

In [152]:
# This basically sums all the values across the 3rd dimension. We will use this to normalise our matrix
N.sum(2,keepdims=True).shape

torch.Size([27, 27, 1])

In [153]:
# We have applied smoothing because there are some cells which might have zero in them (which are impossible to occur according to the dataset)
P=(N+1).float()
P/=P.sum(2,keepdims=True)

In [154]:

g = torch.Generator().manual_seed(2147483647)

for k in range(10):
      out=[]
      ix,iy=0,0

      while True:
          probs=P[ix,iy]
          ix=iy
          iy = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
          out.append(itos[iy])
          
          if iy==0:
           break
      print(''.join(out))



quia.
yu.
quinslyntien.
nolliahi.
ha.
quetony.
uri.
tackareibaidredelingh.
willah.
zulm.


## Now we have a trigram model in which if we give starting 2 characters of a word, we get the most probable next characters till the next character predicted is . This is autoregressive model

******

## We will now improve our model by tuning the weigths to that the words produced at more meaningful

In [155]:
logprobs=0
count=0

for w in words:
    # Adding start and end tokens to denote start and end of a word
    chs=['.']+list(w)+['.']
    for c1,c2,c3 in zip(chs,chs[1:],chs[2:]):
        ch1=stoi[c1]
        ch2=stoi[c2]
        ch3=stoi[c3]
        #N[ch1,ch2]+=1
        probs=P[ch1,ch2,ch3]
        lp=torch.log(probs)
        logprobs+=lp
        count+=1
        #print(f'{ch1}{ch2}{ch3}: {probs:.4f}: {lp:.4f}')
print(f'{logprobs=}')
nll=-logprobs
# Where nll is the negative log likelihood because we want low loss to be good but log likelihood says low value of loss is bad hence negating log likelihood
print(f'{nll=}')
# The below is average negative log likelihood. This would be our loss function generally.
print(f'{nll/count}')

logprobs=tensor(-410414.9688)
nll=tensor(410414.9688)
2.092747449874878


Exercise 1 solution: The min loss observed in trigram model was 2.092 whereas the min loss observed in bigram model was 2.45 so trigram model improved over bigram model.

****

We will now use neural networks to do the same as above

In [156]:
g = torch.Generator().manual_seed(2147483647)
W=torch.randn((27*2,27),generator=g,requires_grad=True)

In [157]:
W.shape

torch.Size([54, 27])

In [158]:
W.dtype

torch.float32

In [159]:
# creating training set i.e having predictors and target in xs and ys

xs=[]
ys=[]

for word in words[:1]:
  chars=['.']+list(word)+['.']

  for f,s,t in zip(chars,chars[1:],chars[2:]):
    ch1=stoi[f]
    ch2=stoi[s]
    ch3=stoi[t]
    xs.append([ch1,ch2])
    ys.append(ch3)

In [160]:
xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [161]:
xs

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1]])

In [162]:
import torch.nn.functional as F

new_xs=F.one_hot(xs,num_classes=27)

In [163]:
# Flattens the inside lists and it essentially concatenates one_hot representations for both input values of a trigram
new_xs=(new_xs.reshape(4,54)).float()

In [164]:
# Where each row will give us the activations of 27 neurons for each example as in the 0th row is giving us activations of each of the 27 neurons and the one which gives the heighest prob will be chosen.
(new_xs@W)[0]

tensor([ 0.5352,  0.2418, -0.2616, -1.0364,  0.4752, -0.3721, -0.2249, -0.4349,
        -0.6773,  1.1168,  1.6238,  1.3033,  0.3456,  0.5469,  2.9616,  1.7805,
         0.8366, -0.8746,  2.0334,  0.3803,  0.6542, -0.8347, -1.6931,  1.4028,
        -1.6675,  0.6432,  1.0764], grad_fn=<SelectBackward0>)

In [165]:

logits = new_xs @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
-probs[torch.arange(4), ys].log().mean()

tensor(4.0953, grad_fn=<NegBackward0>)

In [166]:
probs[torch.arange(4), ys]

tensor([0.0240, 0.0074, 0.0417, 0.0104], grad_fn=<IndexBackward0>)

In [167]:
torch.arange(4)

tensor([0, 1, 2, 3])

In [168]:
(counts/(counts.sum(1,keepdims=True)))[0].sum()#.shape

tensor(1., grad_fn=<SumBackward0>)

In [169]:
counts

tensor([[ 1.7078,  1.2735,  0.7699,  0.3547,  1.6084,  0.6893,  0.7986,  0.6473,
          0.5080,  3.0550,  5.0724,  3.6814,  1.4128,  1.7279, 19.3293,  5.9326,
          2.3085,  0.4170,  7.6398,  1.4628,  1.9236,  0.4340,  0.1840,  4.0667,
          0.1887,  1.9026,  2.9342],
        [ 2.2475,  4.8279,  1.1039,  6.7569,  6.9147,  0.3263,  1.0807,  0.5787,
          0.9320,  0.4274, 10.9643,  8.6845,  0.5825,  0.7380,  0.0906,  0.8456,
          1.3473,  0.4381,  6.5185,  0.4579,  3.2748,  6.3326,  7.2101,  0.0643,
          0.4045,  0.7889, 26.1892],
        [ 1.7007,  3.1414,  1.5148,  3.0331,  1.6480,  0.1593,  2.1359,  0.3030,
          8.2787,  0.1639,  0.8219, 21.0960,  0.4684,  0.7675,  0.0864,  0.2122,
          1.9079,  0.8272,  3.2527,  0.0873,  0.9850,  1.1654,  1.3427,  0.3423,
          2.9067, 12.0535,  4.9325],
        [ 0.7626,  5.7160,  7.3716,  1.3682, 14.5994,  3.2246,  1.3039,  0.4050,
         12.8516,  1.3124,  0.0477,  4.7639,  0.1943,  1.0369,  0.8931,  2.3045

In [170]:
logits.shape

torch.Size([4, 27])

*****

### Doing above for the entire dataset

In [171]:
# creating training set i.e having predictors and target in xs and ys

xs=[]
ys=[]

for word in words:
  chars=['.']+list(word)+['.']

  for f,s,t in zip(chars,chars[1:],chars[2:]):
    ch1=stoi[f]
    ch2=stoi[s]
    ch3=stoi[t]
    xs.append([ch1,ch2])
    ys.append(ch3)

xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [172]:
xs.nelement()

392226

In [173]:
len(xs[0])

2

In [174]:
W=torch.randn((27*2,27),requires_grad=True)

In [175]:
xs=torch.tensor(xs)

<ipython-input-175-4665cfed8f51>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xs=torch.tensor(xs)


In [176]:
import torch.nn.functional as F

new_xs=F.one_hot(xs, num_classes=27)

In [177]:
new_xs.shape

torch.Size([196113, 2, 27])

In [178]:
new_xs=new_xs.reshape(196113,-1).float()

In [179]:
ys.shape

torch.Size([196113])

In [180]:
# We get activations for each of the trigram from each of the 27 neurons
(new_xs@W).shape

torch.Size([196113, 27])

In [181]:
# Calculating the loss and doing backprop
log_counts=(new_xs@W) # This is the log counts
counts=log_counts.exp() # This is the counts
probs=counts/counts.sum(1,keepdims=True) # This is probability
nll=-probs[torch.arange(196113),ys].log().mean()

In [182]:
nll


tensor(4.2833, grad_fn=<NegBackward0>)

In [183]:
W.grad=None
nll.backward()
W.data += -0.1 * W.grad

****

### Doing the optimisation for some iteration and putting it all together

In [184]:
xs=[]
ys=[]

for word in words:
  chars=['.']+list(word)+['.']

  for f,s,t in zip(chars,chars[1:],chars[2:]):
    ch1=stoi[f]
    ch2=stoi[s]
    ch3=stoi[t]
    xs.append([ch1,ch2])
    ys.append(ch3)
    

xs=torch.tensor(xs)
ys=torch.tensor(ys)


# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2, 27), generator=g, requires_grad=True)

In [185]:
xs.shape

torch.Size([196113, 2])

In [186]:
F.one_hot(xs,num_classes=27).shape

torch.Size([196113, 2, 27])

In [187]:
# gradient descent
for k in range(200):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  xenc=xenc.reshape(196113,-1)
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -55 * W.grad

4.1959710121154785
3.3125147819519043
3.0034217834472656
2.840965747833252
2.7417449951171875
2.672031879425049
2.61826753616333
2.5762722492218018
2.5418875217437744
2.5140926837921143
2.490588665008545
2.4712564945220947
2.4543848037719727
2.4403278827667236
2.427609443664551
2.4169716835021973
2.4069724082946777
2.3986923694610596
2.3905720710754395
2.3840174674987793
2.3772475719451904
2.37200927734375
2.3662304878234863
2.362025022506714
2.356982946395874
2.3536040782928467
2.349114179611206
2.346407651901245
2.3423373699188232
2.340183734893799
2.3364367485046387
2.3347418308258057
2.3312506675720215
2.3299386501312256
2.3266549110412598
2.325665235519409
2.322554588317871
2.3218369483947754
2.318873405456543
2.318387746810913
2.315552234649658
2.3152644634246826
2.3125417232513428
2.3124241828918457
2.3098015785217285
2.309831380844116
2.307297706604004
2.307455062866211
2.305001974105835
2.3052709102630615
2.302889823913574
2.3032562732696533
2.3009395599365234
2.30139255523681

In [188]:
W.dtype

torch.float32

In [189]:
F.one_hot(torch.tensor([0,0]),num_classes=27).reshape(1,-1).shape

torch.Size([1, 54])

In [190]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  iy= random.randint(0,26)
  while True:
    
    
    xenc = F.one_hot(torch.tensor([ix,iy]), num_classes=27).float()
    xenc=xenc.reshape(1,-1)
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    ix=iy
    iy = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[iy])
    if iy == 0:
      break
  print(''.join(out))

rpia.
ruanileyloryecn.
arillais.
aa.
eelyeny.


The min loss which we observed using nn was 2.26 while as the min possible loss which we observed using the counts method is 2.092.

****

Exercise 2: Split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [191]:
# First task is to split the dataset into train, validation and test set
# train on bigram model
# train on trigram model
# evaluate on dev and test split

# bigram first

xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)






In [192]:
from sklearn.model_selection import train_test_split

In [193]:
trainval_xs,test_xs,trainval_ys,test_ys=train_test_split(xs,ys,random_state=42,test_size=0.1,shuffle=True)

In [194]:
# Train and validation data
len(trainval_xs),len(trainval_ys)

(205331, 205331)

In [195]:
# test data
len(test_xs),len(test_ys)

(22815, 22815)

In [196]:
# Splitting into train and validation data

xs_train,xs_val,ys_train,ys_val=train_test_split(trainval_xs,trainval_ys,random_state=42,test_size=0.1,shuffle=True)

In [197]:
xs_val.shape

torch.Size([20534])

In [198]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [199]:
# gradient descent
for k in range(100):
  
  # forward pass
  xenc = F.one_hot(xs_train, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(ys_train[0]), ys_train].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -13 * W.grad

3.7934956550598145
3.0204880237579346
2.928511381149292
2.886056900024414
2.8667685985565186
2.8571183681488037
2.8504347801208496
2.846463918685913
2.8434154987335205
2.841400146484375
2.839768648147583
2.8386080265045166
2.8376553058624268
2.836942195892334
2.836354970932007
2.8359110355377197
2.8355348110198975
2.835238218307495
2.8349928855895996
2.8347935676574707
2.834624767303467
2.8344855308532715
2.8343665599823
2.834265947341919
2.834181547164917
2.8341081142425537
2.834043025970459
2.83398699760437
2.833937883377075
2.8338944911956787
2.833855628967285
2.8338215351104736
2.8337903022766113
2.833761692047119
2.8337364196777344
2.8337132930755615
2.833691358566284
2.8336706161499023
2.8336522579193115
2.833635091781616
2.833618402481079
2.8336033821105957
2.8335886001586914
2.8335747718811035
2.833561897277832
2.8335490226745605
2.8335373401641846
2.8335254192352295
2.833514451980591
2.833503484725952
2.83349347114563
2.8334832191467285
2.8334734439849854
2.8334639072418213
2.

### This is the minimum loss we could find by changing the learning rate

In [200]:
## Evaluating on the validation and test set

def evaluate_model(weight,x,yval):
  with torch.no_grad():
    xenc = F.one_hot(x, num_classes=27).float() 
    if len(list(xenc.shape))==3:
      xenc=xenc.reshape(x.shape[0],-1)
    logits = xenc @ weight
    counts = logits.exp() 
    probs = counts / counts.sum(1, keepdims=True) 
    loss = -probs[torch.arange(yval.shape[0]), yval].log().mean()
  return loss


In [201]:
eval_loss=evaluate_model(W,xs_val,ys_val)

In [202]:
eval_loss

tensor(3.6322)

In [203]:
test_loss=evaluate_model(W,test_xs,test_ys)

In [204]:
test_loss

tensor(3.6252)

### The test loss and validation loss for bigram are nearly comparable

### Doing the same for trigram

In [205]:
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2,ch3 in zip(chs, chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append([ix1,ix2])
    ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [206]:
# Splitting into (train+val) and test set
trainval_xs_tri,test_xs_tri,trainval_ys_tri,test_ys_tri=train_test_split(xs,ys,random_state=42,test_size=0.1,shuffle=True)

In [207]:
# Splitting into train and validation dataset

xs_train_tri,xs_val_tri,ys_train_tri,ys_val_tri=train_test_split(trainval_xs_tri,trainval_ys_tri,random_state=42,test_size=0.1,shuffle=True)

In [208]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2, 27), generator=g, requires_grad=True)

In [209]:
xs_train_tri.shape[0]

158850

In [210]:
# gradient descent
for k in range(300):
  
  # forward pass
  xenc = F.one_hot(xs_train_tri, num_classes=27).float() # input to the network: one-hot encoding
  xenc=xenc.reshape(xs_train_tri.shape[0],-1)
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(ys_train_tri.shape[0]), ys_train_tri].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

4.19407320022583
3.364851236343384
3.0501224994659424
2.8792054653167725
2.7746260166168213
2.7020156383514404
2.646439790725708
2.602390766143799
2.566504716873169
2.5368309020996094
2.5119502544403076
2.490886688232422
2.4728596210479736
2.457282781600952
2.443681001663208
2.4316964149475098
2.4210433959960938
2.411505699157715
2.4029104709625244
2.3951237201690674
2.3880343437194824
2.381554365158081
2.3756086826324463
2.3701348304748535
2.365079879760742
2.360398530960083
2.3560516834259033
2.352006673812866
2.3482322692871094
2.3447046279907227
2.341400623321533
2.3383002281188965
2.335386037826538
2.3326430320739746
2.33005690574646
2.3276150226593018
2.3253061771392822
2.323120355606079
2.3210487365722656
2.3190829753875732
2.3172152042388916
2.315438985824585
2.3137478828430176
2.31213641166687
2.310598850250244
2.3091306686401367
2.307727575302124
2.3063852787017822
2.3050999641418457
2.3038687705993652
2.302687406539917
2.301553726196289
2.300464630126953
2.299417734146118
2.

In [211]:
eval_loss_tri=evaluate_model(W,xs_val_tri,ys_val_tri)
test_loss_tri=evaluate_model(W,test_xs_tri,test_ys_tri)

In [212]:
eval_loss_tri,test_loss_tri

(tensor(2.2442), tensor(2.2375))

My trigram model obviously performs better than bigram. The reason i might be having lower test and validation loss is because i have removed regularisation(general practice) during evaluation phase. During training, the regularisation is inflating the loss.

****

Exercise 3: Use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?




**Note: I should have had a function which i could have run many times but now in this setting, i will have to write above could again and again**

Making a function now

In [213]:
def dataset_trigram():
  xs, ys = [], []
  for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2,ch3 in zip(chs, chs[1:],chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]
      xs.append([ix1,ix2])
      ys.append(ix3)
  xs = torch.tensor(xs)
  ys = torch.tensor(ys)
  return xs,ys

In [214]:
x_tensor,y_tensor=dataset_trigram()

In [215]:
xs_trainval,xs_test,ys_trainval,ys_test=train_test_split(x_tensor,y_tensor,random_state=42,test_size=0.1,shuffle=True)

In [216]:
xs_train,xs_val,ys_train,ys_val=train_test_split(xs_trainval,ys_trainval,random_state=42,test_size=0.1,shuffle=True)

In [217]:
def train_trigram(xs_train,ys_train):
    # initialize the 'network'
  g = torch.Generator().manual_seed(2147483647)
  W = torch.randn((27*2, 27), generator=g, requires_grad=True)

    # gradient descent
  for k in range(300):
    
    # forward pass
    xenc = F.one_hot(xs_train, num_classes=27).float() # input to the network: one-hot encoding
    xenc=xenc.reshape(xs_train.shape[0],-1)
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(ys_train.shape[0]), ys_train].log().mean() + (1e-15)*(W**2).mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()
    
    # update
    W.data += -50 * W.grad

  return loss.item(),W
    

In [218]:
l,W_train=train_trigram(xs_train,ys_train)

4.184372901916504
3.3568389415740967
3.042738914489746
2.8721823692321777
2.7678630352020264
2.69545841217041
2.640033006668091
2.59609055519104
2.560276746749878
2.5306482315063477
2.505793809890747
2.4847426414489746
2.4667177200317383
2.4511356353759766
2.4375243186950684
2.4255244731903076
2.4148523807525635
2.4052915573120117
2.39667010307312
2.3888540267944336
2.3817331790924072
2.3752191066741943
2.3692378997802734
2.36372709274292
2.3586337566375732
2.353912830352783
2.3495264053344727
2.345440149307251
2.341625213623047
2.338055372238159
2.334709882736206
2.3315675258636475
2.328611373901367
2.3258259296417236
2.323197603225708
2.320713758468628
2.3183629512786865
2.316135883331299
2.3140227794647217
2.312016010284424
2.310107707977295
2.308290958404541
2.3065595626831055
2.304908037185669
2.303331136703491
2.3018240928649902
2.300382137298584
2.299001455307007
2.2976787090301514
2.2964096069335938
2.295191526412964
2.2940211296081543
2.292895793914795
2.2918131351470947
2.290

In [219]:
l

2.2453463077545166

In [220]:
evaluate_model(W_train,xs_val,ys_val)

tensor(2.2429)

In [221]:
evaluate_model(W_train,xs_test,ys_test)

tensor(2.2360)

I found that the regularisation parameter had to be decreased quite a lot to have comparable results in training,validation. The test loss is lesser than train and validation losses

*****

Exercises 4: E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [222]:
W.shape

torch.Size([54, 27])

In [223]:
F.one_hot(xs_train[:1],num_classes=27)

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0]]])

In [224]:
xs_train[:2],ys_train[:2]

(tensor([[12,  5],
         [ 0, 20]]), tensor([14,  1]))

In [225]:
# This will be the activation 
activation_for_second_across_27_neurons=W[0]+W[20]

In [226]:
h=F.one_hot(xs_train[:2],num_classes=27)

In [227]:
h

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
          0, 0, 0, 0]]])

In [228]:
h.reshape(2,-1)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0]])

In [229]:
(h.reshape(2,-1).float()@W)

tensor([[ 2.8816,  1.1855, -1.2450, -0.3418, -0.2719,  2.1016, -1.3596, -1.2842,
         -0.2921,  1.7929, -1.3506, -0.2122,  1.1643,  0.5749,  2.9728,  0.1881,
         -1.0421, -2.5379,  1.5135,  1.3063,  0.6913, -1.2894,  0.5245, -0.5295,
          0.0726,  2.0374, -0.1776],
        [-2.4900,  4.3650, -1.1868, -0.5876, -0.7969,  3.4873, -1.6074, -2.4537,
          3.3500,  3.0842, -2.1027, -1.8474,  1.7279, -0.4016, -1.0850,  3.8921,
         -2.3495, -2.0927,  3.1016,  0.0634,  2.0082,  2.1951,  0.1874, -0.3798,
         -2.0396,  2.7364,  1.9588]], grad_fn=<MmBackward0>)

In [230]:
W[12]

tensor([ 1.5924,  1.6930, -0.8967,  0.1349, -0.7252,  1.8565, -1.0747, -0.7751,
         0.4439,  1.9663, -0.4620, -0.0523, -1.0233, -0.4285,  0.8220,  1.0321,
        -0.2697, -0.7843,  0.1371,  0.2453, -0.1357,  0.5407, -0.6223,  1.0016,
         0.5405,  1.4884, -0.0439], grad_fn=<SelectBackward0>)

In [231]:
W[27+5]

tensor([ 1.2892, -0.5075, -0.3483, -0.4767,  0.4534,  0.2451, -0.2848, -0.5091,
        -0.7360, -0.1735, -0.8886, -0.1599,  2.1876,  1.0034,  2.1508, -0.8440,
        -0.7724, -1.7536,  1.3764,  1.0610,  0.8269, -1.8301,  1.1468, -1.5311,
        -0.4679,  0.5490, -0.1337], grad_fn=<SelectBackward0>)

In [232]:
li=[]
li.append((W[12]+W[27+5].view(1,27)))
li.append((W[0]+W[27+20]).view(1,27))


In [233]:
li

[tensor([[ 2.8816,  1.1855, -1.2450, -0.3418, -0.2719,  2.1016, -1.3596, -1.2842,
          -0.2921,  1.7929, -1.3506, -0.2122,  1.1643,  0.5749,  2.9728,  0.1881,
          -1.0421, -2.5379,  1.5135,  1.3063,  0.6913, -1.2894,  0.5245, -0.5295,
           0.0726,  2.0374, -0.1776]], grad_fn=<AddBackward0>),
 tensor([[-2.4900,  4.3650, -1.1868, -0.5876, -0.7969,  3.4873, -1.6074, -2.4537,
           3.3500,  3.0842, -2.1027, -1.8474,  1.7279, -0.4016, -1.0850,  3.8921,
          -2.3495, -2.0927,  3.1016,  0.0634,  2.0082,  2.1951,  0.1874, -0.3798,
          -2.0396,  2.7364,  1.9588]], grad_fn=<ViewBackward0>)]

In [234]:
torch.cat(li).shape

torch.Size([2, 27])

In [235]:
def dataset_trigram():
  xs, ys = [], []
  for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2,ch3 in zip(chs, chs[1:],chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]
      xs.append([ix1,ix2])
      ys.append(ix3)
  xs = torch.tensor(xs)
  ys = torch.tensor(ys)
  return xs,ys

In [236]:
x_tensor,y_tensor=dataset_trigram()

In [237]:
xs_trainval,xs_test,ys_trainval,ys_test=train_test_split(x_tensor,y_tensor,random_state=42,test_size=0.1,shuffle=True)

In [238]:
def train_trigram(xs_train,ys_train):
    # initialize the 'network'
  g = torch.Generator().manual_seed(2147483647)
  W = torch.randn((27*2, 27), generator=g, requires_grad=True)

    # gradient descent
  for k in range(300):
    
    # forward pass
    li=[]
    for p in xs_train:
      li.append(W[p[0].item()]+W[27+p[1].item()].view(1,27))
    logits=torch.cat(li)
    # xenc = F.one_hot(xs_train, num_classes=27).float() # input to the network: one-hot encoding
    #xenc=xenc.reshape(xs_train.shape[0],-1)
    #logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(ys_train.shape[0]), ys_train].log().mean() + (1e-15)*(W**2).mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()
    
    # update
    W.data += -50 * W.grad

  return loss.item(),W

In [239]:
l,W_train=train_trigram(xs_train,ys_train)

4.184372901916504
3.356835126876831
3.042736530303955
2.8721816539764404
2.767862558364868
2.695457935333252
2.6400327682495117
2.596090316772461
2.560276746749878
2.5306482315063477
2.505793809890747
2.4847426414489746
2.4667177200317383
2.4511356353759766
2.4375240802764893
2.4255247116088867
2.4148523807525635
2.4052915573120117
2.39667010307312
2.3888537883758545
2.3817331790924072
2.3752191066741943
2.3692378997802734
2.36372709274292
2.3586337566375732
2.353912830352783
2.3495264053344727
2.345440149307251
2.3416249752044678
2.338055372238159
2.334709882736206
2.3315672874450684
2.328611373901367
2.3258261680603027
2.323197603225708
2.320713758468628
2.3183629512786865
2.316135883331299
2.314023017883301
2.3120157718658447
2.310107469558716
2.308290958404541
2.3065595626831055
2.304908037185669
2.303330898284912
2.3018240928649902
2.300382137298584
2.299001455307007
2.2976787090301514
2.2964096069335938
2.295191526412964
2.2940211296081543
2.292895555496216
2.2918131351470947
2.2

This above method performs same as what we did earlier